<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-477 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 1 - Redes Neuronales y *Deep Learning* </H3>
<H3 align='center'>  Jorge Portilla / John Rodriguez </H3>

<hr style="height:2px;border:none"/>

In [1]:
import pandas as pd
import keras
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
import tensorflow as tf
from keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta
from keras.callbacks import Callback
from sklearn.metrics import mean_squared_error as mse
%matplotlib inline

Using TensorFlow backend.


<a id="tercero"></a>
## 3. Entendimiento de imágenes de personas

El problema de inferir ciertas características de una persona a través de una foto de ella puede resultar bastante dificil incluso para nosotros, como por ejemplo de qué país es, la emoción que expresa, la edad que tiene, o el género. La automatización de este proceso para que máquinas logren identificar ciertas características de una persona puede ser algo crucial para el futuro desarrollo de Inteligencia Artificial.


<img src="https://i.imgur.com/6B072GE.jpg" width="60%" height="20%" />


En esta actividad trabajaremos con unos datos (imágenes) con la tarea de predecir la **edad** (*target value*) de la persona en la imagen. Los datos con corresponden a 3640 imágenes de Flickr de rostros de personas, pero, debido a que trabajamos con redes *feed forward*, se trabajará con representaciones de características extraídas. Para ésto necesitará descargar los datos del siguiente __[link](http://chenlab.ece.cornell.edu/people/Andy/ImagesOfGroups.html)__ en el extracto de *ageGenderClassification* o a través de la consola Unix.
```
wget http://chenlab.ece.cornell.edu/projects/ImagesOfGroups/ageGenderClassification.zip
```

Se trabajará con archivos *.mat* que pueden ser cargados de la siguiente manera:
```python
import scipy.io as sio
sio.loadmat("file.mat")
```

Para descripción sobre las columnas están en el archivo readme a través del siguiente __[link](http://chenlab.ece.cornell.edu/projects/ImagesOfGroups/README.txt)__ o a través de la consola Unix:
```
wget http://chenlab.ece.cornell.edu/projects/ImagesOfGroups/README.txt
```

### a) Cargue los datos dos dataset de entrenamiento y de pruebas ¿Cuántos datos hay en cada conjunto?

In [2]:
import scipy.io as sio
mat_train = sio.loadmat("./eventrain.mat")
mat_test = sio.loadmat("./eventest.mat")
data_train= mat_train["trcoll"][0][0]
data_test= mat_test["tecoll"][0][0]
row=len(data_train)
columns=len(data_train[0])
row2=len(data_test)
columns2=len(data_test[0])
print("La cantidad de datos en el conjunto de entrenamiento es ",(columns*row))
print("La cantidad de datos en el conjunto de test es ",(columns2*row2))

La cantidad de datos en el conjunto de entrenamiento es  38500
La cantidad de datos en el conjunto de test es  11550


### b) Eliga cuál representación utilizará para trabajar los datos y entregárselos como *input* al modelo neuronal denso. Además extraiga las etiquetas del problema. Describa los datos utilizados.

Para el problema de clasificación se utilizo la representación de genFeat, donde se tienen 14 caracteristicas de entrada. Estos serian los datos para ser presentados como entrada al modelo neuronal denso. Los nombres de cada una de estas caracteristicas se describen a continuación.

In [4]:
genFeat_train = data_train[0]  #it can be used as representation: contextual features
genFeat_test = data_test[0]
ageClass_train = data_train[1]
ageClass_test = data_test[1]
faceGist_train = data_train[4]
faceGist_test = data_test[4]
ffcoefs_test = data_test[3]
ffcoefs_train = data_train[3]

x_test = pd.DataFrame(genFeat_test, columns=["xcenter", "ycenter", "minSpanningTreeDegree", "SizeRelativeToNeighbor", "PosX", "PosY", "neiAngle", "myAngle",
                              "SizeRelAverage", "x of [x y] position relative average.", "y of [x y] position relative average.", "Size Relative to Planar FaceFit.", "Nearest Neighbor Gender   (NOT USED IN CVPR 09)",
                             "NEarest Neighbor Agebin   (NOT USED IN CVPR 09)"])
x_train = pd.DataFrame(genFeat_train, columns=["xcenter", "ycenter", "minSpanningTreeDegree", "SizeRelativeToNeighbor", "PosX", "PosY", "neiAngle", "myAngle",
                              "SizeRelAverage", "x of [x y] position relative average.", "y of [x y] position relative average.", "Size Relative to Planar FaceFit.", "Nearest Neighbor Gender   (NOT USED IN CVPR 09)",
                             "NEarest Neighbor Agebin   (NOT USED IN CVPR 09)"])
print(x_test)

      xcenter  ycenter  minSpanningTreeDegree  SizeRelativeToNeighbor  \
0        87.0     35.0                    1.0                1.118034   
1        32.0     60.0                    1.0                1.012220   
2        41.0     65.0                    2.0                0.987927   
3        29.0     33.0                    1.0                0.895533   
4        38.0     31.0                    2.0                1.090062   
5        78.0     34.0                    2.0                0.894427   
6        60.0     35.0                    2.0                1.000000   
7        53.0     32.0                    2.0                1.000000   
8        43.0     35.0                    2.0                0.917379   
9        27.0     32.0                    2.0                0.848875   
10       42.0     28.0                    3.0                1.199251   
11       37.0     19.0                    1.0                1.126793   
12       37.0     34.0                    3.0      

### c) Defina y entrene una modelo de red neuronal *feed forward* para la inferencia de la edad de la persona a través de la representación escogida. Intente llegar a un *mse* menor a 100 en el conjunto de pruebas. Recuerde que **NO** puede seleccionar modelos a través del conjunto de pruebas. Visualice sus resultados si estima conveniente.


### Problema como tarea de clasificación

In [25]:
x_test = pd.DataFrame(ffcoefs_test)
x_train = pd.DataFrame(ffcoefs_train)
y_train = ageClass_train
y_test = ageClass_test

y_train_class = np.copy(y_train)
y_test_class = np.copy(y_test)
#Traducción a clases
classes = np.array([1, 5, 10, 16, 28, 51, 75])
for i in range(7):
    y_train_class[y_train_class == classes[i]] = i
    y_test_class[y_test_class == classes[i]] = i

y_train_onehot = keras.utils.to_categorical(y_train_class,num_classes=7)#etiquetas categóricas
y_test_onehot = keras.utils.to_categorical(y_test_class,num_classes=7)#etiquetas categóricas

ffw_network = keras.models.Sequential()
ffw_network.add(Dense(units = 256, kernel_initializer='glorot_uniform', input_dim = x_train.shape[1] ,activation = "sigmoid"))
ffw_network.add(Dense(units = 256 ,kernel_initializer='glorot_uniform', activation = "sigmoid"))
ffw_network.add(Dense(units = 256 ,kernel_initializer='glorot_uniform', activation = "sigmoid"))
ffw_network.add(Dense(units = 256 ,kernel_initializer='glorot_uniform', activation = "sigmoid"))
ffw_network.add(Dense(units = 7, kernel_initializer='glorot_uniform', activation = "softmax"))

ffw_network.compile(optimizer=SGD(lr=0.01), loss='categorical_crossentropy', metrics = [keras.metrics.categorical_accuracy])

history = ffw_network.fit(x_train, y_train_onehot, epochs = 50, batch_size=32, verbose =1, validation_data=(x_test, y_test_onehot))

Train on 3500 samples, validate on 1050 samples
Epoch 1/50
3500/3500 [==============================] - 2s 581us/step - loss: 1.9548 - categorical_accuracy: 0.1457 - val_loss: 1.9591 - val_categorical_accuracy: 0.1429
Epoch 2/50
3500/3500 [==============================] - 1s 366us/step - loss: 1.9526 - categorical_accuracy: 0.1357 - val_loss: 1.9521 - val_categorical_accuracy: 0.1429
Epoch 3/50
3500/3500 [==============================] - 1s 349us/step - loss: 1.9526 - categorical_accuracy: 0.1426 - val_loss: 1.9495 - val_categorical_accuracy: 0.1429
Epoch 4/50
3500/3500 [==============================] - 1s 358us/step - loss: 1.9518 - categorical_accuracy: 0.1366 - val_loss: 1.9490 - val_categorical_accuracy: 0.1429
Epoch 5/50
3500/3500 [==============================] - 1s 423us/step - loss: 1.9518 - categorical_accuracy: 0.1460 - val_loss: 1.9544 - val_categorical_accuracy: 0.1429
Epoch 6/50
3500/3500 [==============================] - 1s 338us/step - loss: 1.9504 - categorical_acc

Epoch 49/50
3500/3500 [==============================] - 1s 393us/step - loss: 1.9513 - categorical_accuracy: 0.1374 - val_loss: 1.9526 - val_categorical_accuracy: 0.1429
Epoch 50/50
3500/3500 [==============================] - 1s 363us/step - loss: 1.9533 - categorical_accuracy: 0.1291 - val_loss: 1.9510 - val_categorical_accuracy: 0.1429


In [26]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
print(ffw_network.evaluate(x_test, y_test_onehot))
y_aprox_class = ffw_network.predict_classes(x_test)

y_aprox = np.copy(y_aprox_class)
for i in range(7):
    y_aprox[y_aprox == i] = classes[i]
#print(y_aprox)
#print(y_test)
print("El error cuadratico medio es:",mse(y_aprox, y_test))

1050/1050 [==============================] - 0s 123us/step
[1.951013677687872, 0.14285714285714285]
El error cuadratico medio es: 637.7142857142857


### Problema como tarea de Regresión

In [11]:
x_test = pd.DataFrame(ffcoefs_test)
x_train = pd.DataFrame(ffcoefs_train)
y_train = ageClass_train
y_test = ageClass_test

scaler = StandardScaler().fit(x_train)
x_train_scaled = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)
x_test_scaled = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)


ffw2 = keras.models.Sequential()
ffw2.add(Dense(256, kernel_initializer='uniform',input_dim=x_train_scaled.shape[1], activation='relu'))
ffw2.add(Dense(256, kernel_initializer='uniform',activation='relu'))
ffw2.add(Dense(256, kernel_initializer='uniform',activation='relu'))
ffw2.add(Dense(1, kernel_initializer='uniform',activation='linear'))
ffw2.compile(loss='mse', optimizer = "rmsprop", metrics = ["accuracy"])
history = ffw2.fit(x_train_scaled, y_train, epochs = 50, verbose =1, validation_data=(x_test, y_test))

Train on 3500 samples, validate on 1050 samples
Epoch 1/50
3500/3500 [==============================] - 2s 578us/step - loss: 511.2256 - acc: 0.0200 - val_loss: 906.6819 - val_acc: 0.0333
Epoch 2/50
3500/3500 [==============================] - 1s 305us/step - loss: 397.8805 - acc: 0.0251 - val_loss: 854.8810 - val_acc: 0.0495
Epoch 3/50
3500/3500 [==============================] - 1s 355us/step - loss: 388.1403 - acc: 0.0337 - val_loss: 892.0030 - val_acc: 0.0657
Epoch 4/50
3500/3500 [==============================] - 1s 347us/step - loss: 377.1154 - acc: 0.0343 - val_loss: 887.0383 - val_acc: 0.0724
Epoch 5/50
3500/3500 [==============================] - 1s 374us/step - loss: 364.4534 - acc: 0.0311 - val_loss: 771.8360 - val_acc: 0.0048
Epoch 6/50
3500/3500 [==============================] - 1s 375us/step - loss: 349.0865 - acc: 0.0294 - val_loss: 736.7531 - val_acc: 0.0657
Epoch 7/50
3500/3500 [==============================] - 1s 340us/step - loss: 334.3504 - acc: 0.0646 - val_loss: